### ISC catalog (International Seismological Center)

In [1]:
import pandas as pd
import numpy as np
import requests

I use **ISC FDSNWS event Web Service**. 
You can check http://www.isc.ac.uk/fdsnws/event/1/ for more information.

| min latitude | max latitude | min longitude | max longitude | start time (yyyy-mm-dd) | end time (yyyy-mm-dd) | format |
| --- | --- | --- | --- | --- | --- | --- |
| 70 | 80 | 100 | 150 | any | 2020-01-01 | xml |



In [2]:
response = requests.get('http://isc-mirror.iris.washington.edu/fdsnws/event/1/query?minlatitude=70&maxlatitude=80&minlongitude=100&maxlongitude=150&endtime=2020-01-01T00:00:00&catalog=ISC&format=xml&nodata=404')

**Let's make our data look properly**

In [3]:
from xml.etree import ElementTree
from contextlib import suppress
 
 
# get_text_helper helps in getting nested text values from tree
def get_text_helper(element, paths, namespaces):
    try:
        for path in paths:
            element = element.find(path, namespaces=namespaces)
        return element.text
    except AttributeError:
        return None
 
 
def parse_xml(text):
    ns = dict(qml="http://quakeml.org/xmlns/quakeml/1.2")
    et = ElementTree.fromstring(text)
    for params in et.getchildren():
        info = params.find("qml:creationInfo", namespaces=ns)
        description = params.find("qml:description", namespaces=ns)
        comment = params.find("qml:comment", namespaces=ns)
        for event in params.findall("qml:event", namespaces=ns):
#           origin_id = event.find("qml:preferredOriginID", namespaces=ns).text
            kind = event.find("qml:type", namespaces=ns).text
            origin = event.find("qml:origin", namespaces=ns)
            time = get_text_helper(origin, ("qml:time", "qml:value"), ns)
            latitude = float(get_text_helper(origin, ("qml:latitude", "qml:value"), ns))
            longitude = float(get_text_helper(origin, ("qml:longitude", "qml:value"), ns))
            depth = None
            magnitude = None
            magtype = get_text_helper(event, ("qml:magnitude", "qml:type"), ns)
            # Ignore error when trying convert to float.
            with suppress(TypeError):
                depth = float(get_text_helper(origin, ("qml:depth", "qml:value"), ns))
                magnitude = float(get_text_helper(event, ("qml:magnitude", "qml:mag", "qml:value"), ns))
            yield dict(
                kind=kind,
                time=time,
                latitude=latitude,
                longitude=longitude,
                depth=depth,
                magnitude=magnitude,
                magtype=magtype
            )

In [4]:
isc_data = pd.DataFrame(data=parse_xml(response.text))

C:\Users\Home\Anaconda3\envs\seismic\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [5]:
isc_data.head(3)

,kind,time,latitude,longitude,depth,magnitude,magtype
0,earthquake,1909-04-10T18:46:54Z,77.5000,128.0000,35000.0,6.75,MS
1,earthquake,1912-04-13T02:39:42Z,80.0000,100.0000,35000.0,5.60,MS
2,earthquake,1918-11-30T06:48:46.86Z,70.5397,130.5124,15000.0,6.40,MS


In [6]:
datetime = np.array(isc_data.time)
# get date
date = [datetime[i].split('T')[0] for i in range(len(datetime))]
# get time
time = [datetime[i].split('T')[1].split('Z')[0].split('.')[0] for i in range(len(datetime))]

In [7]:
isc_data = isc_data.rename(columns={'kind': 'date'})
isc_data['date'] = date
isc_data['time'] = time

In [8]:
# convert depth from m to km
isc_data['depth'] = [isc_data.depth[i] / 1000 for i in range(len(isc_data))]
# exclude events with no magnitude
isc_data = isc_data[isc_data['magnitude'].notnull()]

**Here, we have a table with basic info.
We load the data to isc_data.txt. So, we can use that info for further applications**

In [9]:
isc_data.head(3)

,date,time,latitude,longitude,depth,magnitude,magtype
0,1909-04-10,18:46:54,77.5000,128.0000,35.0,6.75,MS
1,1912-04-13,02:39:42,80.0000,100.0000,35.0,5.60,MS
2,1918-11-30,06:48:46,70.5397,130.5124,15.0,6.40,MS


In [10]:
isc_data.to_csv('isc_data.txt', index=False)

**ISF is the IASPEI approved standard format for the exchange of parametric seismological data (hypocentres, magnitudes, phase arrivals, moment tensors, etc.).**

| min latitude | max latitude | min longitude | max longitude | start time (yyyy-mm-dd) | end time (yyyy-mm-dd) | format |
| --- | --- | --- | --- | --- | --- | --- |
| 70 | 80 | 100 | 150 | any | 2020-01-01 | isf |

In [11]:
response = requests.get('http://isc-mirror.iris.washington.edu/fdsnws/event/1/query?minlatitude=70&maxlatitude=80&minlongitude=100&maxlongitude=150&endtime=2020-01-01T00:00:00&catalog=ISC&format=isf&nodata=404')

**So, we have the possibility to get more information. Let's keep it in mind for further applications.**

### United States Geological Survey (NEIC)
You can check https://earthquake.usgs.gov/fdsnws/event/1/ for more information.

| min latitude | max latitude | min longitude | max longitude | start time (yyyy-mm-dd) | end time (yyyy-mm-dd) | 
| --- | --- | --- | --- | --- | --- | 
| 70 | 80 | 100 | 150 | 1970-01-01 | 2020-01-01 | 

In [12]:
response = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/query?format=text&starttime=1970-01-01&endtime=2020-01-01&minlatitude=70&minlongitude=100&maxlatitude=80&maxlongitude=150&orderby=time')

In [13]:
usgs_data = response.text.split('\n')

for line in range(len(usgs_data)):
    usgs_data[line] = usgs_data[line].split('|')

usgs_data = pd.DataFrame(usgs_data[1:-1], columns=usgs_data[0])

In [14]:
usgs_data = usgs_data.rename(columns={'#EventID': 'date', 'Time': 'time', 'Latitude': 'latitude', 'Longitude': 'longitude',
                                      'Depth/km': 'depth', 'Magnitude': 'magnitude', 'MagType':'magtype'})

In [15]:
usgs_data.head(2)

,date,time,latitude,longitude,depth,Author,Catalog,Contributor,ContributorID,magtype,magnitude,MagAuthor,EventLocationName
0,us600050jx,2019-08-04T13:22:56.174,71.4671,138.4826,10,us,us,us,us600050jx,mb,4.3,us,"193km NNE of Ust'-Kuyga, Russia"
1,us70003ntt,2019-05-21T03:45:11.918,75.493,110.1978,10,us,us,us,us70003ntt,mb,4.8,us,"167km NNW of Nordvik, Russia"


In [16]:
datetime = np.array(usgs_data.time)
# get date
date = [datetime[i].split('T')[0] for i in range(len(datetime))]
# get time
time = [datetime[i].split('T')[1].split('.')[0] for i in range(len(datetime))]

usgs_data['date'] = date
usgs_data['time'] = time

In [17]:
usgs_data = usgs_data[['date', 'time', 'latitude', 'longitude', 'depth', 'magnitude', 'magtype']]

In [18]:
usgs_data.head(3)

,date,time,latitude,longitude,depth,magnitude,magtype
0,2019-08-04,13:22:56,71.4671,138.4826,10,4.3,mb
1,2019-05-21,03:45:11,75.493,110.1978,10,4.8,mb
2,2019-05-13,00:43:33,79.4127,123.9058,10,5.4,mww


**Here, we have a table with basic info.
We load the data to isc_data.txt. So, we can use that info for further applications**

In [19]:
usgs_data.to_csv('usgs_data.txt', index=False)